In [2]:
# %% [markdown]
# # IFCManager - Ejemplo de Uso Completo
# 
# Este notebook demuestra las funcionalidades principales del módulo IFCManager para análisis de archivos IFC.

# %% [markdown]
# ## 1. Importación de Librerías

# %%
import sys
import os
import logging
import pandas as pd
from pathlib import Path

# Agregar el directorio src al path
sys.path.append('src')

# Importar directamente desde el paquete principal
from ifc_manager import IFCManager, IFCParser, ModelVisualizer

# Configurar logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

# %% [markdown]
# ## 2. Inicialización del IFCManager

# %%
# Ruta al archivo IFC (ajustar según tu archivo)
ifc_file_path = "sample_models/Edificio_modelo.ifc"  # Cambiar por tu archivo IFC

# Inicializar el manager
manager = IFCManager()

# %% [markdown]
# ## 3. Carga y Análisis del Archivo IFC

# %%
try:
    # Cargar archivo IFC
    success = manager.load_ifc(ifc_file_path)
    
    if success:
        print("✅ Archivo IFC cargado exitosamente")
        
        # Extraer elementos estructurales
        elements = manager.extract_structural_elements()
        print(f"📊 Elementos estructurales extraídos: {len(elements)}")
        
    else:
        print("❌ Error al cargar el archivo IFC")
        # Crear datos de ejemplo para demostración
        print("🔧 Creando datos de ejemplo para demostración...")
        elements = create_sample_data()
        manager.structural_elements = elements
        
except Exception as e:
    print(f"❌ Error: {e}")
    print("🔧 Creando datos de ejemplo para demostración...")
    elements = create_sample_data()
    manager.structural_elements = elements

# %% [markdown]
# ## 4. Función para Crear Datos de Ejemplo

# %%
def create_sample_data():
    """Crea datos de ejemplo cuando no hay archivo IFC disponible."""
    return {
        'beam-001': {
            'id': 'beam-001',
            'name': 'Viga Principal',
            'type': 'IfcBeam',
            'geometry': {
                'type': 'line',
                'start': [0, 0, 3],
                'end': [6, 0, 3],
                'vertices': [[0,0,3], [6,0,3], [6,0.3,3], [0,0.3,3]],
                'triangles': [[0,1,2], [0,2,3]]
            },
            'material': {
                'Concreto': {
                    'name': 'Concreto',
                    'type': 'material',
                    'properties': {'fc': 25, 'density': 2400}
                }
            },
            'profile': {
                'shape': 'rectangular',
                'height': 0.4,
                'width': 0.3
            },
            'properties': {}
        },
        'column-001': {
            'id': 'column-001',
            'name': 'Columna C1',
            'type': 'IfcColumn',
            'geometry': {
                'type': 'line',
                'start': [0, 0, 0],
                'end': [0, 0, 3],
                'vertices': [[0,0,0], [0,0,3], [0.3,0,3], [0.3,0,0]],
                'triangles': [[0,1,2], [0,2,3]]
            },
            'material': {
                'Concreto': {
                    'name': 'Concreto',
                    'type': 'material',
                    'properties': {'fc': 25, 'density': 2400}
                }
            },
            'profile': {
                'shape': 'rectangular',
                'height': 0.3,
                'width': 0.3
            },
            'properties': {}
        },
        'slab-001': {
            'id': 'slab-001',
            'name': 'Losa Nivel 1',
            'type': 'IfcSlab',
            'geometry': {
                'type': 'mesh',
                'vertices': [[0,0,3], [6,0,3], [6,4,3], [0,4,3]],
                'triangles': [[0,1,2], [0,2,3]]
            },
            'material': {
                'Concreto': {
                    'name': 'Concreto',
                    'type': 'material',
                    'properties': {'fc': 25, 'density': 2400}
                }
            },
            'profile': None,
            'properties': {'thickness': 0.15}
        }
    }

# %% [markdown]
# ## 5. Análisis de Elementos por Tipo

# %%
# Contar elementos por tipo
element_types = {}
for elem_id, elem_data in manager.structural_elements.items():
    elem_type = elem_data['type']
    element_types[elem_type] = element_types.get(elem_type, 0) + 1

print("📈 Resumen de elementos por tipo:")
for elem_type, count in element_types.items():
    print(f"  • {elem_type}: {count}")

# %% [markdown]
# ## 6. Análisis de Materiales

# %%
# Extraer información de materiales
materials_summary = {}
for elem_id, elem_data in manager.structural_elements.items():
    if elem_data.get('material'):
        for mat_name, mat_data in elem_data['material'].items():
            if mat_name not in materials_summary:
                materials_summary[mat_name] = {
                    'count': 0,
                    'elements': [],
                    'properties': mat_data.get('properties', {})
                }
            materials_summary[mat_name]['count'] += 1
            materials_summary[mat_name]['elements'].append(elem_data['name'])

print("🏗️ Resumen de materiales:")
for mat_name, mat_info in materials_summary.items():
    print(f"  • {mat_name}: {mat_info['count']} elementos")
    if mat_info['properties']:
        print(f"    Propiedades: {list(mat_info['properties'].keys())}")

# %% [markdown]
# ## 7. Crear DataFrame para Análisis

# %%
# Convertir elementos a DataFrame para análisis
elements_data = []
for elem_id, elem_data in manager.structural_elements.items():
    row = {
        'ID': elem_data['id'],
        'Name': elem_data['name'],
        'Type': elem_data['type'],
        'Has_Geometry': 'geometry' in elem_data,
        'Has_Material': 'material' in elem_data,
        'Has_Profile': 'profile' in elem_data and elem_data['profile'] is not None
    }
    
    # Agregar información de perfil si existe
    if elem_data.get('profile'):
        profile = elem_data['profile']
        row['Profile_Shape'] = profile.get('shape', 'N/A')
        row['Profile_Height'] = profile.get('height', 0)
        row['Profile_Width'] = profile.get('width', 0)
    
    # Agregar material principal
    if elem_data.get('material'):
        materials = list(elem_data['material'].keys())
        row['Primary_Material'] = materials[0] if materials else 'N/A'
    
    elements_data.append(row)

df_elements = pd.DataFrame(elements_data)
print("📊 DataFrame de elementos estructurales:")
print(df_elements.to_string(index=False))

# %% [markdown]
# ## 8. Funciones de Consulta

# %%
# Demostrar funciones de consulta del parser
if hasattr(manager, 'parser') and manager.parser:
    parser = manager.parser
    
    # Buscar elementos por tipo
    beams = parser.get_elements_by_type('IfcBeam')
    columns = parser.get_elements_by_type('IfcColumn')
    slabs = parser.get_elements_by_type('IfcSlab')
    
    print(f"🔍 Consultas específicas:")
    print(f"  • Vigas encontradas: {len(beams)}")
    print(f"  • Columnas encontradas: {len(columns)}")
    print(f"  • Losas encontradas: {len(slabs)}")
    
    # Mostrar detalles de un elemento específico
    if beams:
        beam_id = list(beams.keys())[0]
        beam_data = parser.get_structural_element(beam_id)
        print(f"\n📋 Detalles de viga {beam_id}:")
        print(f"  • Nombre: {beam_data.get('name', 'Sin nombre')}")
        print(f"  • Tipo: {beam_data.get('type')}")
        if beam_data.get('profile'):
            profile = beam_data['profile']
            print(f"  • Perfil: {profile.get('shape')} - {profile.get('height')}x{profile.get('width')}")

# %% [markdown]
# ## 9. Visualización 3D

# %%
try:
    # Crear visualización
    fig = manager.visualize_model()
    
    # Mostrar la figura
    fig.show()
    print("✅ Visualización 3D generada exitosamente")
    print("💡 La visualización se abrirá en una nueva ventana del navegador")
    
except Exception as e:
    print(f"❌ Error en visualización: {e}")
    print("💡 Asegúrate de tener Plotly instalado: pip install plotly")

# %% [markdown]
# ## 10. Estadísticas Finales

# %%
# Generar estadísticas finales
total_elements = len(manager.structural_elements)
elements_with_geometry = sum(1 for elem in manager.structural_elements.values() 
                            if 'geometry' in elem)
elements_with_material = sum(1 for elem in manager.structural_elements.values() 
                           if 'material' in elem)

print("📈 Estadísticas finales del análisis:")
print(f"  • Total de elementos: {total_elements}")
print(f"  • Elementos con geometría: {elements_with_geometry}")
print(f"  • Elementos con material: {elements_with_material}")
print(f"  • Tipos de elementos únicos: {len(element_types)}")
print(f"  • Materiales únicos: {len(materials_summary)}")

# Calcular porcentajes
if total_elements > 0:
    geom_percent = (elements_with_geometry / total_elements) * 100
    mat_percent = (elements_with_material / total_elements) * 100
    print(f"  • Completitud de geometría: {geom_percent:.1f}%")
    print(f"  • Completitud de materiales: {mat_percent:.1f}%")

print("\n🎉 Análisis completado exitosamente!")

# %% [markdown]
# ## Conclusiones
# 
# Este ejemplo demuestra las capacidades principales del módulo IFCManager:
# 
# 1. **Carga de archivos IFC** - Lectura y validación de archivos
# 2. **Extracción de elementos** - Identificación de elementos estructurales
# 3. **Análisis de propiedades** - Materiales, perfiles y geometría
# 4. **Consultas específicas** - Filtrado por tipo y búsquedas
# 5. **Visualización 3D** - Representación gráfica del modelo
# 6. **Análisis estadístico** - Resúmenes y métricas del modelo
# 
# El módulo proporciona una interfaz sencilla para trabajar con modelos BIM y extraer información relevante para análisis estructural.

INFO: Loading IFC file: sample_models/Edificio_modelo.ifc
INFO: IFC file loaded successfully with 818 products
INFO: Extracting structural elements...
INFO: Found 456 elements of type IfcBeam


✅ Archivo IFC cargado exitosamente


INFO: Found 255 elements of type IfcColumn
INFO: Found 17 elements of type IfcSlab
INFO: Found 53 elements of type IfcWall
INFO: Found 0 elements of type IfcFooting
INFO: Found 0 elements of type IfcPile
INFO: Found 0 elements of type IfcMember
INFO: Extracted 781 structural elements in total
INFO: Extracting geometry for structural elements...
INFO: Geometry extraction completed


❌ Error: 'NoneType' object is not subscriptable
🔧 Creando datos de ejemplo para demostración...


NameError: name 'create_sample_data' is not defined